In [1]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
#df = pd.read_csv("noAudible/Features_294too299hz.csv")
#df2 = pd.read_csv("noAudible/Features_10too15hz.csv")
df = pd.read_csv("Frequenzbänder/Features_20too30hz.csv")
#df4 = pd.read_csv("noAudible/Features_20too25hz.csv")
#df5 = pd.read_csv("noAudible/Features_25too30hz.csv")
#df = pd.read_csv("noAudible/Features_5too10hz.csv")

#frames =[df1,df2,df3,df4,df5]
#df = pd.concat(frames)



#df = pd.read_csv("Features_all_no_audible.csv")
#df = df[(df.modeID != 'audible')]
df

C:\Users\Magomed\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


,channel_1_mav,channel_1_mav2,channel_1_rms,channel_1_iav,channel_1_ssi,channel_1_var,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
0,0.004279,0.004127,0.005494,14.328795,0.101104,0.000030,3.852517,14.328795,0.001150,283,...,0.001413,290,293,0,92.792793,123.294129,1,1001,10010001,audible
1,0.003746,0.003500,0.004804,8.444557,0.052011,0.000023,2.239453,8.444557,0.000994,188,...,0.001082,202,201,0,38.839286,118.270205,1,1001,10010002,audible
2,0.004339,0.004186,0.006225,17.602685,0.157226,0.000039,4.683781,17.602685,0.001154,346,...,0.001535,348,360,0,98.762376,132.911507,1,1001,10010003,audible
3,0.004039,0.004038,0.005441,16.821881,0.123302,0.000030,4.402088,16.821881,0.001057,344,...,0.001488,362,372,0,64.337349,130.442117,1,1001,10010004,audible
4,0.004321,0.004360,0.005616,20.315120,0.148285,0.000032,5.380354,20.315120,0.001145,399,...,0.001566,396,407,0,119.616205,134.898919,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.005931,0.005686,0.008357,13.855172,0.163134,0.000070,3.726452,13.855172,0.001595,204,...,0.000354,206,205,0,51.724138,130.042572,8,8019,80190246,whisper
4796,0.003710,0.003696,0.004892,7.067770,0.045581,0.000024,1.885116,7.067770,0.000990,162,...,0.000293,164,167,0,41.269841,122.075749,8,8019,80190247,whisper
4797,0.005368,0.005136,0.007287,12.098546,0.119683,0.000053,3.181595,12.098546,0.001412,188,...,0.000390,189,198,0,56.250000,127.745907,8,8019,80190248,whisper
4798,0.003923,0.003633,0.005575,6.716901,0.053218,0.000031,1.731969,6.716901,0.001012,143,...,0.000293,148,148,0,42.352941,123.572835,8,8019,80190249,whisper


In [2]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [3]:
def lda_user1():
    train_ids = [2,3,4,5,6,7,8]
    test_ids = [1]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
def lda_user2():
    train_ids = [1,3,4,5,6,7,8]
    test_ids = [2]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user3():
    train_ids = [1,2,4,5,6,7,8]
    test_ids = [3]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 
    
def lda_user4():
    train_ids = [1,2,3,5,6,7,8]
    test_ids = [4]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
        
def lda_user5():
    train_ids = [1,2,3,4,6,7,8]
    test_ids = [5]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score   
    
def lda_user6():
    train_ids = [1,2,3,4,5,7,8]
    test_ids = [6]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user7():
    train_ids = [1,2,3,4,5,6,8]
    test_ids = [7]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score    
    
def lda_user8():
    train_ids = [1,2,3,4,5,6,7]
    test_ids = [8]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 

In [4]:
def lda_users():
    a = lda_user1()
    b = lda_user2()
    c = lda_user3()
    d = lda_user4()
    e = lda_user5()
    f = lda_user6()
    g = lda_user7()
    h = lda_user8()
    scores = np.array([a,b,c,d,e,f,g,h])
    return scores

In [5]:
def multiple():
    scores = np.array([])
    for i in range (31):
        test = lda_users()
        scores = np.concatenate([scores,test],axis=0)
    print("Durchschnitt:")
    print((scores.mean()))
    print(scores.std())
    

In [6]:
multiple()

Accuracy for session 19 is 0.4022222222222222
Accuracy for session 19 is 0.3288888888888889
Accuracy for session 19 is 0.3333333333333333
Accuracy for session 19 is 0.47
Accuracy for session 19 is 0.48
Accuracy for session 19 is 0.58
Accuracy for session 19 is 0.5266666666666666
Accuracy for session 19 is 0.3476190476190476
Accuracy for session 19 is 0.4022222222222222
Accuracy for session 19 is 0.3288888888888889
Accuracy for session 19 is 0.3333333333333333
Accuracy for session 19 is 0.47
Accuracy for session 19 is 0.48
Accuracy for session 19 is 0.58
Accuracy for session 19 is 0.5266666666666666
Accuracy for session 19 is 0.3476190476190476
Accuracy for session 19 is 0.4022222222222222
Accuracy for session 19 is 0.3288888888888889
Accuracy for session 19 is 0.3333333333333333
Accuracy for session 19 is 0.47
Accuracy for session 19 is 0.48
Accuracy for session 19 is 0.58
Accuracy for session 19 is 0.5266666666666666
Accuracy for session 19 is 0.3476190476190476
Accuracy for session 1

Accuracy for session 19 is 0.3333333333333333
Accuracy for session 19 is 0.47
Accuracy for session 19 is 0.48
Accuracy for session 19 is 0.58
Accuracy for session 19 is 0.5266666666666666
Accuracy for session 19 is 0.3476190476190476
Accuracy for session 19 is 0.4022222222222222
Accuracy for session 19 is 0.3288888888888889
Accuracy for session 19 is 0.3333333333333333
Accuracy for session 19 is 0.47
Accuracy for session 19 is 0.48
Accuracy for session 19 is 0.58
Accuracy for session 19 is 0.5266666666666666
Accuracy for session 19 is 0.3476190476190476
Accuracy for session 19 is 0.4022222222222222
Accuracy for session 19 is 0.3288888888888889
Accuracy for session 19 is 0.3333333333333333
Accuracy for session 19 is 0.47
Accuracy for session 19 is 0.48
Accuracy for session 19 is 0.58
Accuracy for session 19 is 0.5266666666666666
Accuracy for session 19 is 0.3476190476190476
Accuracy for session 19 is 0.4022222222222222
Accuracy for session 19 is 0.3288888888888889
Accuracy for session 1